In [1]:
#imports
import requests
import pandas as pd
import json
import time
import networkx as nx

In [2]:
class ReleaseNode:
    """
    Represents a node in a networkx diagram.with basic attributes and functionalities(TBD).

    Attributes:
        release_name (str): Name of the music release.
        release_mbid (str): MusicBrainz ID.
        release_credits (list): List of credits on the music release.
    """
    def __init__(self, release_title, release_mbid, release_credit):
        """
        Initializes the Release_Node object.

        Args:
            release_name (str): Name of the music release.
            release_mbid (str): MusicBrainz ID.
            release_credits (list): List of credits on the music release.
        """
        self.release_title = release_title
        self.release_mbid = release_mbid
        self.release_credit = release_credit

    def __str__(self):
        return f"A release with title {self.name} and mbid {self.release_mbid}"

In [3]:
def search_recording(artist_name, release_title):
    """
    Retrieves all release groups for a given artist using their MusicBrainz ID.

    Args:
        artist_name (str): Name of the artist.
        release_title (str): Name of the Release

    Returns:
        list: The MBID for the release. This is the initial node for the network.
    """
    base_url = "https://musicbrainz.org/ws/2/release/"
    params = {
        "query": f"artist:\"{artist_name}\" AND release:\"{release_title}\"",
        "fmt": "json"
    }
    headers = {"User-Agent": "MusicProject/1.0 (bittah.pupil@proton.me)"} # Replace with your app info

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status() # Raise an exception for bad status codes
    data = response.json()
    
    if data and data.get('releases'):
        # Return the MBID of the first matching recording
        return data['releases'][0]['id']
    return None

# Example usage:
#artist = "Chico Hamilton"
#album = "Peregrinations"
#release_mbid = search_recording(artist, album)
#print(f"Recording MBID for '{album}' by '{artist}': {release_mbid}")

In [4]:
async def get_credits_for_recording(release_mbid):
    """
    Retrieves all the credits for a given release using the releases MusicBrainz ID.

    Args:
        release_mbid (str): MusicBrainz ID for the release.

    Returns:
        list: A list of all credited personnel (including artist) on the release based on the provided release mbid.
        This list includes the name of the credit, the role of the credit, and the mbid for the credit.
    """
    base_url = f"https://musicbrainz.org/ws/2/release/{release_mbid}"
    params = {
        "inc": "artist-credits+labels+recordings+recording-level-rels+work-rels+work-level-rels+artist-rels + release-groups",  # Include artist relationships
        "fmt": "json"
    }
    headers = {"User-Agent": "MusicProject/1.0 (bittah.pupil@proton.me)"}

    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status()
    data = response.json()

    credits = []
    
    if data and data.get('artist-credit'):
       for track in data['media'][0]['tracks']:
            for credit in track['recording']['relations']:
                if credit['attributes'] == []:
                    attributes = 'Not Found'
                else:
                    attributes = credit['attributes']
                credits.append((track['number'], track['title'], credit.get('artist', {}).get('name', 'Not Found'), attributes, credit.get('artist', {}).get('id', 'Not Found')))
            for credit in track['artist-credit']:
                credits.append((track['number'], track['title'], credit.get('name', 'Not Found'), credit.get('attributes', 'Not Found'), credit.get('artist', {}).get('id', 'Not Found')))
    if data['relations']:
       for credit in data['relations']:
           trackNum = 'Not Found'
           trackTitle = 'Not Found'
           if credit['attributes'] == [] or not credit['attributes']:
               attributes = 'Not Found'
           else:
               attributes = credit['attributes']
           credits.append((trackNum, trackTitle, credit.get('artist', {}).get('name', 'Not Found'), attributes, credit.get('artist', {}).get('id', 'Not Found')))

    creditNames = []
    creditList = []
    for credit in credits:
        if credit[2] not in creditNames:
            creditNames.append(credit[2])
            creditList.append((credit[2],credit[3], credit[4]))
    
    return creditList

In [5]:
async def get_artist_releases(artist_mbid):
    """
    Retrieves all release groups for a given artist using their MusicBrainz ID.

    Args:
        artist_mbid (str): The MusicBrainz ID of the artist.

    Returns:
        list: A list of dictionaries, each representing a release group.
              Returns an empty list if no releases are found or an error occurs.
    """
    base_url = "https://musicbrainz.org/ws/2/release-group/"
    studio_albums = []
    #releases = []
    limit = 100  # Maximum results per request
    offset = 0

    while True:
        params = {
            "artist": artist_mbid,
            "type": "album",
            #"primary-type":"album",
            "fmt": "json",
            "limit": limit,
            "offset": offset
        }
        headers = {
            "User-Agent": "MusicProject/1.0 (bittah.pupil@proton.me)" # Replace with your app name and contact email
        }

        try:
            response = requests.get(base_url, params=params, headers=headers)
            response.raise_for_status()  # Raise an exception for HTTP errors
            data = response.json()

            release_groups = data.get("release-groups", [])
            for rg in release_groups:
                # Filter out live albums and compilations
                secondary_types = rg.get("secondary-types", [])
            
                if "Live" not in secondary_types and "Compilation" not in secondary_types:
                    album_info = {
                        "title": rg.get("title"),
                        "mbid": rg.get("id"),
                        "first_release_date": rg.get("first-release-date", "Unknown"),
                        "primary_type": rg.get("primary-type"),
                        "secondary_types": secondary_types
                    }
                    studio_albums.append(album_info)
            
            #releases.extend(release_groups)

            # Check if there are more results to fetch
            if len(release_groups) < limit:
                break
            offset += limit
            time.sleep(1)

        except requests.exceptions.RequestException as e:
            print(f"Error fetching data: {e}")
            break
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON response: {e}")
            break

    #return releases
    return studio_albums

In [6]:
# provide initial inputs and retrieve release_mbid for the release
artist = "Earl Klugh"
album = "One On One"
release_mbid = search_recording(artist, album)
print(f"Recording MBID for '{album}' by '{artist}': {release_mbid}")

G = nx.Graph()

# create initial nodelist and linkslist

# <--- Consider starting while loop here for looping through a certain number of degrees of seperation

# retrieve credit personnel for the initial release
if release_mbid:
    albumCredits = await get_credits_for_recording(release_mbid)
    print(albumCredits)
    print("\n\n")
else:
    print(f"Recording for '{album}' by '{artist}' not found.")

testList = []

# extract each credits releases
for credit in albumCredits:
    releases = await get_artist_releases(credit[2])
    print(f"Credit Name: {credit[0]}:")
    #print(releases)
    #print("\n")
    #for release in releases:
    #  Create Node (Release Node)
    #  Add note to node list
    #  Create link in linklist
    for release in releases:
        releaseNodeObject = ReleaseNode(release.get("title"), release.get("mbid"), credit[0])
        testList.append(releaseNodeObject)
for obj in testList:
    print(vars(obj))
# display node diagram (maybe in next) cell

Recording MBID for 'One On One' by 'Earl Klugh': 1609def0-5de3-4b10-be96-24ad6bb2a6f8
[('Bob James', ['keyboard'], 'f4d2fc5a-cacb-47c8-b096-ee73864b1aa8'), ('Neil Jason', ['bass'], '18826a91-40a1-46c2-b0d6-dc3b95ca88ee'), ('Earl Klugh', ['guitar'], '5c44b1b8-07ad-4aca-ac5d-ebdad8e62e12'), ('Hubert Laws', ['flute'], '82208529-b45a-4b3e-92dd-02f6b14c6f18'), ('Ralph MacDonald', ['percussion'], 'b6be6ceb-f776-460f-a145-212fa306baa1'), ('Harvey Mason', ['drums (drum set)'], '1081d9bb-17fa-43cf-8b2e-05f7e6029f70'), ('David Nadien', ['violin'], '512598a0-8a21-4f93-93ca-f2ec75f98a4a'), ('Not Found', 'Not Found', 'Not Found'), ('Eric Gale', ['guitar'], '72323249-ff36-49d2-8925-f6b5a2946e5c'), ('Gary King', ['bass'], 'eba85512-eaf1-41d0-b007-679ae56bec42'), ('Ron Carter', ['bass'], '57db3f59-9c58-4f68-a00e-e044666c4828')]



Credit Name: Bob James:
Credit Name: Neil Jason:
Credit Name: Earl Klugh:
Credit Name: Hubert Laws:
Credit Name: Ralph MacDonald:
Credit Name: Harvey Mason:
Credit Name: Dav